In [41]:
import pickle

import pandas as pd
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [36]:
from sklearn.pipeline import make_pipeline

In [42]:
os.environ["AWS_PROFILE"] = "pana"

TRACKING_HOST_SERVER = 'ec2-35-173-1-88.compute-1.amazonaws.com'
mlflow.set_tracking_uri(f"http://{TRACKING_HOST_SERVER}:5000")

In [43]:
print(f"tracking uri: {mlflow.get_tracking_uri()}")

tracking uri: http://ec2-35-173-1-88.compute-1.amazonaws.com:5000


In [44]:

mlflow.set_experiment("green-taxi-experiment")

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

2022/06/26 22:18:21 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-experiment' does not exist. Creating a new experiment.


In [45]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [46]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [48]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-store/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-store/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-store/2', experiment_id='2', lifecycle_stage='active', name='green-taxi-experiment', tags={}>]